### Shared Word

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
sentence_check = True  # True, False
#twogram_sentence_check = False  # True, False
#threegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/1-Twogram Threegram Shared Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
#Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [ ]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [ ]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

In [ ]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [ ]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [ ]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}_Result{file_ext}.xlsx", index=False)

    #if sentence_check:
    #    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}_Sentence{file_ext}.xlsx", index=False)
    #else:
    #    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}_All{file_ext}.xlsx", index=False)    

#### Concat Result

In [ ]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

In [ ]:
# Option For Frequency
if shared_word_frequency:
    #df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

In [ ]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Result{file_ext}.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)

#if sentence_check:
#    df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Sentence{file_ext}.xlsx")
#    df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
#    df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
#else:
#    df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_All{file_ext}.xlsx")
#    df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
#    df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)

df_shared_twogram

In [ ]:
# Option For Twogram Sentence
#if twogram_sentence_check:
if sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram

In [ ]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_Result{file_ext}.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)


#if sentence_check:
#    df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_Sentence{file_ext}.xlsx")
#    df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
#    df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
#else:
#    df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_All{file_ext}.xlsx")
#    df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
#    df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)

df_shared_threegram

In [ ]:
# Option For Threegram Sentence
#if threegram_sentence_check:
if sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_threegram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

In [ ]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram], axis=1)
df_shared_all

In [ ]:
# Optional Output
#if sentence_check:
#    df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_twogram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_threegram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#else:
#    df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram_All_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_twogram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_All_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_threegram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_All_Result_With_Frequency{file_ext}.xlsx", index=False)

##### Concat Result With Comma

In [ ]:
# \******
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

In [ ]:
df_ety_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
df_ety_prefix_suffix = df_ety_prefix_suffix[["search_word","word"]]
df_ety_prefix_suffix

In [ ]:
if file_ext == "1":
    df_word_not_prefix_suffix = df_word_all
    df_word_not_prefix_suffix["search_word"] = df_word_all["word"]
    df_word_not_prefix_suffix = df_word_not_prefix_suffix["search_word","word"]
    
    df_ety_not_prefix_suffix = df_pair
    df_ety_not_prefix_suffix = df_ety_not_prefix_suffix.rename(columns={"dict_entry_main":"word"})
    df_ety_not_prefix_suffix["search_word"] = df_ety_not_prefix_suffix["word"]
    df_ety_not_prefix_suffix = df_ety_not_prefix_suffix["search_word","word"]
    
    df_all_word = pd.concat([df_word_not_prefix_suffix,df_ety_not_prefix_suffix],axis=0)
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "2":
    df_ety_not_prefix_suffix = df_pair
    df_ety_not_prefix_suffix = df_ety_not_prefix_suffix.rename(columns={"dict_entry_main":"word"})
    df_ety_not_prefix_suffix["search_word"] = df_ety_not_prefix_suffix["word"]
    df_ety_not_prefix_suffix = df_ety_not_prefix_suffix["search_word","word"]

    df_all_word = df_ety_not_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "3":
    df_all_word = pd.concat([df_word_prefix_suffix,df_ety_prefix_suffix],axis=0)
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "4":
    df_all_word = df_ety_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "6":
    df_word_not_prefix_suffix = df_word_all
    df_word_not_prefix_suffix["search_word"] = df_word_all["word"]
    df_word_not_prefix_suffix = df_word_not_prefix_suffix["search_word","word"]

    df_all_word = df_word_not_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word
# /******

In [ ]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [ ]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram")  

In [ ]:
# \*****
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")

In [ ]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()  
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()

In [ ]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram]

In [ ]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all 

In [ ]:
## Option For Word Frequency
#if shared_word_frequency:
#    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
#    df_word_order_join_all.drop_duplicates(inplace=True)
#    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram"]]
#    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
#    df_word_order_join_all.reset_index(drop=True, inplace=True)
#else:
#    pass
#
#df_word_order_join_all

In [ ]:
df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
df_word_order_join_all.drop_duplicates(inplace=True)
df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram"]]
df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
df_word_order_join_all.reset_index(drop=True, inplace=True)
df_word_order_join_all

In [ ]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [ ]:
df_word_all = df_word_all.loc[:,["word","frequency"]]
df_word_all["word"] = df_word_all["word"].apply(lambda x: lower_strip_func(x))
df_word_all

In [ ]:
df_twogram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_all = df_twogram_all.loc[:,["twogram","frequency"]]
df_twogram_all["twogram"] = df_twogram_all["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram_all

In [ ]:
df_threegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram_all = df_threegram_all.loc[:,["threegram","frequency"]]
df_threegram_all["threegram"] = df_threegram_all["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram_all

In [ ]:
df_shared_process_all = df_word_order_join_all
for i in range(len(df_shared_process_all)):
    # column result
    try:
        # column result
        df_two_var = pd.DataFrame(df_shared_process_all.loc[i,"twogram"].split(", "), columns=["twogram"])
        # merge with all
        df_two_var_merge = pd.merge(df_two_var, df_twogram_all, how="left", on="twogram")
        df_two_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_two_var_merge_select = df_two_var_merge.head(word_sample_num)
        # to list value
        df_two_var_list = df_two_var_merge_select["twogram"].to_list()
        # list join
        df_two_var_list_join = ", ".join(df_two_var_list)
        # change value
        df_shared_process_all.loc[i,"twogram"] = df_two_var_list_join
    except:
        pass
    try:
        # column result
        df_three_var = pd.DataFrame(df_shared_process_all.loc[i,"threegram"].split(", "), columns=["threegram"])
        # merge with all
        df_three_var_merge = pd.merge(df_three_var, df_threegram_all, how="left", on="threegram")
        df_three_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_three_var_merge_select = df_three_var_merge.head(word_sample_num)
        # to list value
        df_three_var_list = df_three_var_merge_select["threegram"].to_list()
        # list join
        df_three_var_list_join = ", ".join(df_three_var_list)
        # change value
        df_shared_process_all.loc[i,"threegram"] = df_three_var_list_join
    except:
        pass

df_shared_process_all
# /****

In [ ]:
df_word_order_join_twogram = df_shared_process_all[["word","frequency","twogram"]]
df_word_order_join_threegram = df_shared_process_all[["word","frequency","threegram"]]

In [ ]:
if sentence_check:
    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
else:
    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

In [ ]:
#df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
#df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()

In [ ]:
#dfs = [df_word_order_join_twogram,df_word_order_join_threegram]

In [ ]:
#df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
##df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
#df_word_order_join_all  

In [ ]:
## Option For Word Frequency
#if shared_word_frequency:
#    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
#    df_word_order_join_all.drop_duplicates(inplace=True)
#    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram"]]
#    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
#    df_word_order_join_all.reset_index(drop=True, inplace=True)
#
#    df_word_order_join_twogram = pd.merge(df_word_order_join_twogram,df_word_all, how="inner", on="word")
#    df_word_order_join_twogram.drop_duplicates(inplace=True)
#    df_word_order_join_twogram = df_word_order_join_all.loc[:,["word","frequency","twogram"]]
#    df_word_order_join_twogram.sort_values(by="frequency", inplace=True, ascending=False)
#    df_word_order_join_twogram.reset_index(drop=True, inplace=True)
#
#    df_word_order_join_threegram = pd.merge(df_word_order_join_threegram,df_word_all, how="inner", on="word")
#    df_word_order_join_threegram.drop_duplicates(inplace=True)
#    df_word_order_join_threegram = df_word_order_join_all.loc[:,["word","frequency","threegram"]]
#    df_word_order_join_threegram.sort_values(by="frequency", inplace=True, ascending=False)
#    df_word_order_join_threegram.reset_index(drop=True, inplace=True)
#else:
#    pass
#
#df_word_order_join_all

In [ ]:
#df_word_order_join_all.word.nunique()

In [ ]:
#if sentence_check:
#    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
#    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
#    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
#else:
#    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
#    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
#    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    #destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass